In [176]:
import pandas
import os
import re

In [177]:
SAMPLE = 'sample_0'
SOLVED_FILE_PATH = 'data/output/solved/'
EQUATIONS_FILE_PATH = ''

In [178]:
ground = pandas.read_json('data/input/draw.json')
ground = ground[['lSolutions', 'iIndex']]
ground = ground.set_index('iIndex')
display(len(ground))
ground.head(1)

999

,lSolutions
iIndex,
397760,[2.14285714286]


In [179]:
solved_majority = pandas.read_json('data/output/solved_majority_eqn/solved_majority.jsonl', lines=True)
solved_majority.head(1)

,solved,flag
0,{a: 1155.00000000000},OK


In [180]:
def extract_actual_answer(answer):
    answer = re.sub(r'\s', "", answer)
    answer = re.sub(r'\s|.*####', "", answer)
    return eval(answer)

def remove_leading_zero(number : str):
    return re.sub(r'^0+', '', number)

def extract_chatgpt_answer(answer_text):
    answer_text = answer_text.lower()
    try:
        answers = re.findall(r'the\s*answer\s*is\s*.*', answer_text)[-1]
        numbers = re.findall(r'\d+\.?\d*', answers)
        fractions = re.findall(r'\d+\/\d+', answers)
        answers = numbers + fractions

        to_return = []
        for answer in answers:
            try: to_return.append(round(eval(remove_leading_zero(answer))))
            except: pass
        return set(to_return)
    except:
        answers = re.findall(r'\d+\.?\d*', answer_text)
        if len(answers) == 0: return set()
        return set([eval(answers[0])])
    
def extract_solved(solved):
    answers = re.findall(r'\d+\.?\d*', solved)
    return set([round(eval(answer), 2) for answer in answers])

def grade_response(row, file_name):
    solved = row[f'{file_name}__solved']
    system = row['choices']
    solved = extract_solved(solved)
    system = extract_chatgpt_answer(system)
    chatgpt_answer = extract_chatgpt_answer(row[f'{file_name}__question'])
    actual_answer = set(row['lSolutions'])

    solved = set([round(s, 2) for s in solved])
    system = set([round(s, 2) for s in system])
    actual_answer = set([round(s, 2) for s in actual_answer])
    

    row[f'chatgpt_answer'] = chatgpt_answer
    row[f'solved_answer'] = solved
    row[f'system_answer'] = system

    A = 1 if chatgpt_answer.issubset(actual_answer) > 0 and len(chatgpt_answer) > 0 else 0
    C = 1 if system.issubset(actual_answer) > 0 and len(system) > 0 else 0
    D = 1 if solved.issubset(actual_answer) > 0 and len(solved) > 0 else 0

    row['A. ChatGPT gets it correct from text to final solution'] = A
    row['B. Sympy gets it correct from equations to final solution'] = 1
    row['C. ChatGPT gets it correct from equations to final solution'] = C
    row['D. Gets right answer when ChatGPT transforms text to equations and SymPy solves the equations from chat GPT'] = D

    category_array = [A, C, D]

    offset = 8 if chatgpt_answer.issubset(solved) > 0 and len(chatgpt_answer) > 0 else 0
    row['E. Final answer and equations same'] = 1 if chatgpt_answer.issubset(solved) > 0 and len(chatgpt_answer) > 0 else 0
    if category_array == [0, 0 , 0]: row['category'] = 1 + offset
    if category_array == [0, 0 , 1]: row['category'] = 2 + offset
    if category_array == [0, 1 , 0]: row['category'] = 3 + offset
    if category_array == [0, 1 , 1]: row['category'] = 4 + offset
    if category_array == [1, 0 , 0]: row['category'] = 5 + offset
    if category_array == [1, 0 , 1]: row['category'] = 6 + offset
    if category_array == [1, 1 , 0]: row['category'] = 7 + offset
    if category_array == [1, 1 , 1]: row['category'] = 8 + offset
    
    return row

In [181]:
system = pandas.read_json('data/output/system_equations/sample_0.jsonl', lines=True)
system = system[['question_id', 'choices']]
system['choices'] = system['choices'].apply(lambda row : row[0]['message']['content'])
system = system.set_index('question_id')
system.head(1)

,choices
question_id,
74243,There seems to be only one equation given. Can...


In [182]:
ground = ground.join(system)

for file_path in os.listdir(SOLVED_FILE_PATH):
    solved = pandas.read_json(f"{SOLVED_FILE_PATH}{file_path}", lines=True)
    solved = solved[['question_id', 'question', 'solved']]
    # solved['choices'] = solved['choices'].apply(lambda row : row[0]['message']['content'])
    file_name = file_path.split('.')[0]
    solved = solved.rename(columns={'question': f'{file_name}__question', 'solved': f'{file_name}__solved'})
    solved = solved.set_index('question_id')
    ground = ground.join(solved, how='left')
    ground = ground.apply(lambda row : grade_response(row, file_name), axis=1)

majority_eqn = pandas.read_json('data/output/solve_majority_test/sample_0.jsonl',lines=True)
majority_eqn = majority_eqn[['question_id', 'response', 'majority_equation', 'solved']]
majority_eqn = majority_eqn.rename(columns={'question': f'majority__question', 'response': f'majority__question', 'majority_equation': f'majority__choices', 'solved': f'majority__solved'})
majority_eqn = majority_eqn.set_index('question_id')
ground = ground.join(majority_eqn)
ground = ground.apply(lambda row : grade_response(row, 'majority'), axis=1)

# print(len(ground))
ground = ground[~ground.index.duplicated(keep='first')]
ground.head(3)

,lSolutions,choices,sample_0__question,sample_0__solved,chatgpt_answer,solved_answer,system_answer,A. ChatGPT gets it correct from text to final solution,B. Sympy gets it correct from equations to final solution,C. ChatGPT gets it correct from equations to final solution,...,sample_6__solved,sample_7__question,sample_7__solved,sample_8__question,sample_8__solved,sample_9__question,sample_9__solved,majority__question,majority__choices,majority__solved
iIndex,,,,,,,,,,,,,,,,,,,,,
337,[2.4],Step-by-step solution:\n1/4 + 1/6 = 3/12 + 2/1...,Extract the equations from this response.\nYou...,{a: 2.40000000000000},{2},"{12, 5}","{2, 12, 5}",0,1,0,...,FREE VARIABLE[],Extract the equations from this response.\nYou...,{a: 12/5},Extract the equations from this response.\nYou...,"FREE VARIABLE{a: c/b, d: 12/5}",Extract the equations from this response.\nYou...,"FREE VARIABLE{a: 5/12, b: e*f/6, c: e*f/4, d: ...",Step 1: \nLet x be the number of hours it take...,1/4 + 1/6 = 1/x\n5/12 = 1/x\nx = 12/5,{a: 12/5}
1042,"[9.0, 12.0]",boys+girls=21 --> Equation 1\n3*boys-4*girl...,Extract the equations from this response.\nYou...,"{a: 12, b: 9}","{9, 12}","{9, 12}","{9, 12}",1,1,1,...,"{a: 12, b: 9}",Extract the equations from this response.\nYou...,"{a: 12, b: 9}",Extract the equations from this response.\nYou...,"{a: 12, b: 9}",Extract the equations from this response.\nYou...,"{a: 12, b: 9}",Step 1: \nLet x be the larger number and y be ...,3x = 4y\nx + y = 21,"{a: 9, b: 12}"
1299,"[12.5, 87.5]",Substitute y in the first equation with 7*x fr...,Extract the equations from this response.\nYou...,"{a: 12.5000000000000, b: 87.5000000000000}","{88, 12}",{12.5},"{88, 12}",0,1,0,...,{a: 12.5000000000000},Extract the equations from this response.\nYou...,"{a: 25/2, b: 175/2}",Extract the equations from this response.\nYou...,{a: 12.5000000000000},Extract the equations from this response.\nYou...,"{a: 87.5000000000000, b: 12.5000000000000}","Step 1: \nLet's call the value of the saddle ""...",x + 7x = 100.00\n8x = 100.00\nx = 12.50\n7x = ...,{a: 12.5000000000000}


In [183]:
def label_category(row):
    category = row.name[0]
    return row

values = pandas.DataFrame(ground[[
    'category', 
    'A. ChatGPT gets it correct from text to final solution', 
    'B. Sympy gets it correct from equations to final solution',
    'C. ChatGPT gets it correct from equations to final solution',
    'D. Gets right answer when ChatGPT transforms text to equations and SymPy solves the equations from chat GPT',
    'E. Final answer and equations same'
]].value_counts().sort_index())

values

,,,,,,count
category,A. ChatGPT gets it correct from text to final solution,B. Sympy gets it correct from equations to final solution,C. ChatGPT gets it correct from equations to final solution,D. Gets right answer when ChatGPT transforms text to equations and SymPy solves the equations from chat GPT,E. Final answer and equations same,
1,0,1,0,0,0,157
2,0,1,0,1,0,91
3,0,1,1,0,0,50
4,0,1,1,1,0,135
5,1,1,0,0,0,9
6,1,1,0,1,0,5
7,1,1,1,0,0,29
8,1,1,1,1,0,55
9,0,1,0,0,1,37


In [184]:
import xlsxwriter

writer = pandas.ExcelWriter('data/output/xlsx/categories.xlsx', engine='xlsxwriter')
values.to_excel(writer, sheet_name='List')

workbook = writer.book

wrap_format = workbook.add_format()
wrap_format.set_text_wrap()

list_sheet = writer.sheets['List']
list_sheet.set_column('A:H', 15, wrap_format)
pie_chart = workbook.add_chart({'type': 'pie'})

pie_chart.set_style(10)
pie_chart.set_size({'width': 500, 'height': 300})
pie_chart.set_title({'name': f'ChatGPT stats'})
pie_chart.add_series({
    'name': f'ChatGPT stats',
    'categories': f"=Chart!$A$2:$A$10",
    'values': f"=Chart!$G$2:$G$16",
    'points': [
        {'fill': {'color': '#e7366b'}},
        {'fill': {'color': '#29cc7a'}},
        {'fill': {'color': '#ffc619'}},
    ],
    'data_labels': {'value': True, 'category': True, 'position': 'center', 'percentage': True, 'name_font': {'size': 14}}
})
list_sheet.insert_chart('A20', pie_chart, {'x_offset': 0, 'y_offset': 0})
workbook.close()

C:\Users\nocet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xlsxwriter\workbook.py:1660: UserWarning: Unknown worksheet reference 'Chart' in range 'Chart!$A$2:$A$10' passed to add_series()
  warn("Unknown worksheet reference '%s' in range "
C:\Users\nocet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xlsxwriter\workbook.py:1660: UserWarning: Unknown worksheet reference 'Chart' in range 'Chart!$G$2:$G$16' passed to add_series()
  warn("Unknown worksheet reference '%s' in range "


In [185]:
values

,,,,,,count
category,A. ChatGPT gets it correct from text to final solution,B. Sympy gets it correct from equations to final solution,C. ChatGPT gets it correct from equations to final solution,D. Gets right answer when ChatGPT transforms text to equations and SymPy solves the equations from chat GPT,E. Final answer and equations same,
1,0,1,0,0,0,157
2,0,1,0,1,0,91
3,0,1,1,0,0,50
4,0,1,1,1,0,135
5,1,1,0,0,0,9
6,1,1,0,1,0,5
7,1,1,1,0,0,29
8,1,1,1,1,0,55
9,0,1,0,0,1,37
